In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_normalized = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 
                                                                        'price_change_percentage_7d', 
                                                                        'price_change_percentage_14d', 
                                                                        'price_change_percentage_30d', 
                                                                        'price_change_percentage_60d', 
                                                                        'price_change_percentage_200d', 
                                                                        'price_change_percentage_1y']])

In [ ]:
# Create a DataFrame with the scaled data
df_market_data_normalized = pd.DataFrame(market_data_normalized, columns = ['price_change_percentage_24h', 
                                                                            'price_change_percentage_7d', 
                                                                            'price_change_percentage_14d', 
                                                                            'price_change_percentage_30d', 
                                                                            'price_change_percentage_60d', 
                                                                            'price_change_percentage_200d', 
                                                                            'price_change_percentage_1y'])


# Copy the crypto names from the original data
df_market_data_normalized['coinid'] = df_market_data.index

# Set the coinid column as index
df_market_data_normalized.set_index('coinid', inplace=True)

# Display sample data
df_market_data_normalized.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [ ]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in range(1, 12):
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans = KMeans(n_clusters=k, random_state=0)
# 2. Fit the model to the data using `df_market_data_scaled`
    kmeans.fit(df_market_data_normalized)
# 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve_data = {'k': k, 'inertia': inertia_values}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_curve_data = pd.DataFrame(elbow_curve_data)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_curve = df_elbow_curve_data.hvplot.line(x = 'k', y = 'inertia', title = 'Elbow Curve', xticks = k)
elbow_curve

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Eleven would be the best value for K

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
best_k = KMeans(n_clusters = 11)

In [ ]:
# Fit the K-Means model using the scaled data
best_k.fit(df_market_data_normalized)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
cluster_group = best_k.predict(df_market_data_normalized)

# Print the resulting array of cluster values.
print(cluster_group)

In [ ]:
# Create a copy of the DataFrame
df_market_normalized_predictions = df_market_data_normalized.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_market_normalized_predictions['CryptoCluster'] = cluster_group

# Display sample data
print(df_market_normalized_predictions.head())

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
crypto_scatter = df_market_normalized_predictions.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    color='CryptoCluster', 
    hover_cols=['crypto_names'],
    title='Crypto Clusters with K-means Using the Original Data'
)

crypto_scatter

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_model = pca.fit_transform(df_market_data_normalized)
# View the first five rows of the DataFrame. 
pca_model[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance is equal to 89.5% of the three principal components

In [ ]:
# Create a new DataFrame with the PCA data.
df_pca_data = pd.DataFrame(pca_model, columns=['PCA1', 'PCA2', 'PCA3'])

# Copy the crypto names from the original data
df_pca_data ['coinid'] = df_market_data.index

# Set the coinid column as index
df_pca_data.set_index('coinid', inplace=True)

# Display sample data
df_pca_data.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_pca = list(range(1, 12))

In [ ]:
# Create an empty list to store the inertia values
inertia_values_pca = []

# Create a for loop to compute the inertia with each possible value of k
for k in range(1, 12):
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans_pca = KMeans(n_clusters=k, random_state=0)
# 2. Fit the model to the data using `df_market_data_pca`
    kmeans_pca.fit(df_pca_data)
# 3. Append the model.inertia_ to the inertia list
    inertia_values_pca.append(kmeans_pca.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve_pca = {'k': k_pca, 'inertia': inertia_values_pca}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_curve_pca = pd.DataFrame(elbow_curve_pca)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_chart_pca = df_elbow_curve_pca.hvplot.line(x = 'k', y = 'inertia', title = 'Elbow Curve using PCA Data', xticks = k)
elbow_chart_pca

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** Four would be the best value for K


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:**  This data suggest that a smaller number for the K value would be best meanwhile the larger number in the previous set would be better in that data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
best_k_pca = KMeans(n_clusters = 4)

In [ ]:
# Fit the K-Means model using the PCA data
best_k_pca.fit(df_pca_data)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
cluster_labels_pca = best_k_pca.predict(df_pca_data)

# Print the resulting array of cluster values.
print(cluster_labels_pca)

In [ ]:
# Create a copy of the DataFrame with the PCA data
df_pca_cluster = df_pca_data.copy()

# Add a new column to the DataFrame with the predicted clusters
df_pca_cluster['cluster'] = cluster_labels_pca

# Display sample data
df_pca_cluster.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

# Create a scatter plot with specified columns
crypto_scatter_pca = df_pca_cluster.hvplot.scatter(
    x='PCA1',
    y='PCA2',
    c='cluster',  # Use cluster labels for coloring
    colormap='viridis',  # Choose a colormap for coloring
    hover_cols=['crypto_names'],  # Display crypto names on hover
    title='Crypto Clusters with K-means Using PCA'
)

# Show the scatter plot
crypto_scatter_pca

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
# Define the Elbow curves for both datasets (original and PCA)
elbow_curve_original = df_elbow_curve_data.hvplot.line(
    x='k', y='inertia', label='Original Data', color='red'
)
elbow_curve_pca = df_elbow_curve_pca.hvplot.line(
    x='k', y='inertia', label='PCA Data', color='blue'
)

# Combine the two curves into a single plot
composite_plot = (elbow_curve_original * elbow_curve_pca).opts(
    title='Elbow Curves for Original and PCA Data',
    xlabel='Number of Clusters (K)',
    ylabel='Inertia',
    legend_position='top_right',
)

# Display the composite plot
composite_plot

In [ ]:
# Composite plot to contrast the clusters
# Define the scatter plots for both datasets (original and PCA)
scatter_original = df_market_data_normalized.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    c='cluster',  # Use cluster labels for coloring
    colormap='viridis',  # Choose a colormap for coloring
    hover_cols=['crypto_names'],  # Display crypto names on hover
    label='Original Data',
)
scatter_pca = df_pca_cluster.hvplot.scatter(
    x='PCA1',
    y='PCA2',
    c='cluster',  # Use cluster labels for coloring
    colormap='viridis',  # Choose a colormap for coloring
    hover_cols=['crypto_names'],  # Display crypto names on hover
    label='PCA Data',
)

# Combine the two scatter plots into a single plot
composite_plot_clusters = (scatter_original * scatter_pca).opts(
    title='Crypto Clusters for Original and PCA Data',
    legend_position='top_right',
    width=600, height=400,
)

# Display the composite plot
composite_plot_clusters

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Looking at the visualization from the cluster analysis, we are able to see a clear image of what the data is trying to tell. The data is showing a large cluster between the groups that are around the negative values for both x and y axises.